# Analysis of Shuffling Batches
Shuffling batches refers to the act of shuffling the training data each epoch and producing different batches, as opposed to only shuffling the data once at the beginning and producing an identical set of batches each epoch.

First thing to point out is that using the ReLU activation in regression can cause the model loss blow up sometimes. The analysis is performed with these large values left in the data. Normally I would not consider models would for comparison, but I wish to measure what performance from the first $n$ trained models, rather than the first $n$ successful models. 

Also due to the different loss metrics being used in regression and classification (RMSE & negative log loss) they are hard to compare, e.g. at what point is a RMSE measurement equivalent to a log loss measurement?. For this reason the comparisons containing both learning tasks should be considered carefully.

For hypothesis testing Welch's t-test is used, with a threshold of .001 for the p-value.

## Summary
Shuffling batches seems to benefit generalisation ability the most, and is at times harmful to test performance. 

Some guidelines derived from the analyses:
- Shuffling batches improves validation accuracy.
- Shuffling batches is not beneficial when the goal is to evaluate the ability to learn the data and generalisation is not expected (e.g. for toy datasets such as XOR).
- Shuffling batches is always recommended when working on classification tasks. 

A general piece of advice: 
- Do to not use ReLU on regression tasks unless you are willing to spend more time than usual tuning hyperparameters.

In [57]:
import os

import numpy as np
import pandas as pd
from scipy import stats

In [2]:
os.getcwd()

'/home/anthony/420/notebooks'

In [68]:
df = pd.read_csv('../scripts/results_summary.csv', index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4536 entries, ce2406bc88ceda4e2fde96bf96fdd2d5 to 9591fefef4086cbf060bc6afdcee4eba
Columns: 168 entries, activation_func to val_loss_39
dtypes: bool(1), float64(163), int64(1), object(3)
memory usage: 5.8+ MB


In [44]:
df.describe()

,batch_size,gaussian_noise,learning_rate,momentum,train_loss_00,train_loss_01,train_loss_02,train_loss_03,train_loss_04,train_loss_05,...,val_loss_30,val_loss_31,val_loss_32,val_loss_33,val_loss_34,val_loss_35,val_loss_36,val_loss_37,val_loss_38,val_loss_39
count,4536.000000,4536.000000,4536.000000,4536.000000,4.469000e+03,4.465000e+03,4.474000e+03,4.460000e+03,4.468000e+03,4.480000e+03,...,1.291000e+03,1.291000e+03,1.291000e+03,1.293000e+03,1.284000e+03,1.289000e+03,1.286000e+03,1.287000e+03,1.285000e+03,1.288000e+03
mean,3.238095,0.036667,0.037000,0.466667,6.139973e+148,1.178551e+143,2.240893e+146,1.080856e+143,5.007608e+146,1.060628e+132,...,3.490164e+139,1.192571e+140,6.523714e+144,3.004334e+88,5.270054e+87,2.447946e+133,2.201866e+139,7.127668e+127,1.836580e+128,4.176261e+140
std,7.432443,0.044974,0.044704,0.368219,4.104608e+150,7.788403e+144,1.498695e+148,7.218305e+144,3.347240e+148,7.099082e+133,...,1.254033e+141,4.284966e+141,2.344002e+146,1.079347e+90,1.846660e+89,8.788773e+134,7.896076e+140,2.557036e+129,6.583569e+129,1.498806e+142
min,-1.000000,0.000000,0.001000,0.000000,3.878009e-12,1.887379e-15,6.075696e-14,1.249001e-15,2.439715e-14,7.771561e-16,...,1.205098e-02,4.532566e-03,4.032520e-03,8.639882e-07,2.478734e-02,9.347693e-04,1.913314e-02,1.281576e-02,3.864593e-03,1.995955e-02
25%,-1.000000,0.000000,0.001000,0.000000,2.304792e-01,2.317160e-01,2.297833e-01,2.342466e-01,2.307808e-01,2.303364e-01,...,1.340462e-01,1.338613e-01,1.409296e-01,1.371336e-01,1.328476e-01,1.330931e-01,1.354859e-01,1.379978e-01,1.337317e-01,1.374174e-01
50%,1.000000,0.010000,0.010000,0.500000,5.006268e-01,5.005960e-01,5.009687e-01,5.008767e-01,5.006290e-01,5.009712e-01,...,2.280760e-01,2.075429e-01,2.246615e-01,2.367095e-01,2.146115e-01,2.205804e-01,2.219849e-01,2.380792e-01,2.077034e-01,2.332943e-01
75%,2.000000,0.100000,0.100000,0.900000,6.950951e-01,6.948118e-01,6.947904e-01,6.948717e-01,6.953892e-01,6.954834e-01,...,3.588411e-01,3.450229e-01,3.522622e-01,3.709971e-01,3.579938e-01,3.465480e-01,3.506236e-01,3.654653e-01,3.506442e-01,3.557505e-01
max,32.000000,0.100000,0.100000,0.900000,2.743954e+152,5.203949e+146,1.002447e+150,4.820618e+146,2.237399e+150,4.751615e+135,...,4.505802e+142,1.539609e+143,8.422115e+147,3.881150e+91,6.615518e+90,3.155402e+136,2.831599e+142,9.173309e+130,2.360005e+131,5.379024e+143


In [60]:
df.describe()

,batch_size,gaussian_noise,learning_rate,momentum,train_loss_00,train_loss_01,train_loss_02,train_loss_03,train_loss_04,train_loss_05,...,val_loss_30,val_loss_31,val_loss_32,val_loss_33,val_loss_34,val_loss_35,val_loss_36,val_loss_37,val_loss_38,val_loss_39
count,4536.000000,4536.000000,4536.000000,4536.000000,4.421000e+03,4.408000e+03,4.420000e+03,4.409000e+03,4.416000e+03,4.424000e+03,...,1269.000000,1274.000000,1274.000000,1.276000e+03,1265.000000,1270.000000,1269.000000,1269.000000,1270.000000,1275.000000
mean,3.238095,0.036667,0.037000,0.466667,9.492196e-01,9.856368e-01,9.650471e-01,1.008959e+00,9.500885e-01,9.786358e-01,...,0.662106,0.602162,0.851149,6.383636e-01,0.673178,0.563561,0.570689,0.644398,0.778400,0.717671
std,7.432443,0.044974,0.044704,0.368219,2.551082e+00,3.058140e+00,2.717477e+00,3.221689e+00,2.714989e+00,2.986081e+00,...,3.273116,2.011437,4.774636,2.011593e+00,2.559179,1.751126,1.892626,2.650772,4.349055,2.965329
min,-1.000000,0.000000,0.001000,0.000000,3.878009e-12,1.887379e-15,6.075696e-14,1.249001e-15,2.439715e-14,7.771561e-16,...,0.012051,0.004533,0.004033,8.639882e-07,0.024787,0.000935,0.019133,0.012816,0.003865,0.019960
25%,-1.000000,0.000000,0.001000,0.000000,2.290924e-01,2.293545e-01,2.277254e-01,2.324011e-01,2.281162e-01,2.264769e-01,...,0.133462,0.133300,0.140651,1.369765e-01,0.132286,0.132207,0.134755,0.137504,0.133561,0.136948
50%,1.000000,0.010000,0.010000,0.500000,5.004064e-01,5.004145e-01,5.006733e-01,5.006643e-01,5.004337e-01,5.006053e-01,...,0.215599,0.203238,0.215326,2.264593e-01,0.202375,0.213418,0.216666,0.228667,0.203096,0.229127
75%,2.000000,0.100000,0.100000,0.900000,6.946095e-01,6.943673e-01,6.944147e-01,6.943980e-01,6.948703e-01,6.947385e-01,...,0.340059,0.336998,0.344854,3.642472e-01,0.347973,0.337470,0.341807,0.352550,0.338966,0.345505
max,32.000000,0.100000,0.100000,0.900000,4.329897e+01,9.937854e+01,6.472528e+01,9.880452e+01,9.643732e+01,8.603724e+01,...,85.228314,22.081121,93.338842,2.035628e+01,43.754461,13.815511,29.434711,54.095100,98.826480,58.866476


In [45]:
df.head()

,activation_func,batch_size,clf_type,dataset,gaussian_noise,learning_rate,momentum,shuffle_batches,train_loss_00,train_loss_01,...,val_loss_30,val_loss_31,val_loss_32,val_loss_33,val_loss_34,val_loss_35,val_loss_36,val_loss_37,val_loss_38,val_loss_39
run_id,,,,,,,,,,,,,,,,,,,,,
ce2406bc88ceda4e2fde96bf96fdd2d5,LeakyReLU,1,MLPRegressor,xor,0.0,0.1,0.9,True,5.166794e-01,5.365412e-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
745da0d20037327f586babfb38077b62,LeakyReLU,1,MLPRegressor,xor,0.0,0.1,0.9,False,4.087733e-01,1.385021e-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aeeec51b4e4d9c885bd53d0de5c46b14,LeakyReLU,1,MLPRegressor,xor,0.0,0.1,0.5,True,3.247473e-01,4.423432e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a99a2a1ecd0377d4d4d5c47d3015771b,LeakyReLU,1,MLPRegressor,xor,0.0,0.1,0.5,False,6.579213e-01,3.202256e-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a523da2d87ca23feb5411114cf717710,LeakyReLU,1,MLPRegressor,xor,0.0,0.1,0.0,True,5.498153e+38,1.364288e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
metrics = ['train_loss', 'train_scores', 'val_loss', 'val_scores']

In [199]:
def print_stats(dataframe, name='a'):
    a = dataframe
    print('Summary for %s:' % name)
    print('n: %d' % len(a.dropna()))
    # Do aggregate functions (e.g. mean()) twice to get column-wise and then datafram-wise measures.
    print('min: %.4e - max: %.4e' % (a.min().min(), a.max().max()))
    print('μ ± 2σ: %.4e ± %.4e' % (a.mean().mean(), 2 * a.std().std()))
    print('Quartiles - 1st: %.4e - 2nd: %.4e - 3rd: %.4e' % (a.quantile(0.25).quantile(0.25),
                                                            a.quantile(0.5).quantile(0.5),
                                                            a.quantile(0.75).quantile(0.75)))
     
        
def format_p_value(p_value):
    return str(p_value % 1)[1:5] if p_value > 0.001 else '<.001'
        
def stat_test(masks, test_name, alpha=0.001):    
    assert len(masks) == 2

    print('*' * 80)
    print(test_name)
    print('*' * 80)
    
    for metric in metrics:
        print('#'  * len(metric))
        print(metric)
        print('#'  * len(metric))
    
        a = df[masks[0]].filter(regex=metric + '_\d{2}')
        b = df[masks[1]].filter(regex=metric + '_\d{2}')
    
        print_stats(a, name='a')
        print()

        print_stats(b, name='b')
        print()
        
        print('Welch t-test:')
        t, p = stats.ttest_ind(a.values.ravel(), b.values.ravel(), equal_var=False, nan_policy='omit')
        print('t: %.4f - p: %s' % (t, format_p_value(p)))

In [200]:
shuffles_batch = df['shuffle_batches'] == True
no_shuffling = df['shuffle_batches'] == False
using_regression = df['clf_type'] == 'MLPRegressor'

stat_test(masks=(shuffles_batch & using_regression, no_shuffling & using_regression), 
          test_name='Regression - Shuffling Batches vs. Not Shuffling Batches:')

********************************************************************************
Regression - Shuffling Batches vs. Not Shuffling Batches:
********************************************************************************
##########
train_loss
##########
Summary for a:
n: 1077
min: 7.2164e-16 - max: 2.7440e+152
μ ± 2σ: 6.5877e+147 ± 2.5883e+150
Quartiles - 1st: 1.7842e-01 - 2nd: 3.1288e-01 - 3rd: 5.0199e-01

Summary for b:
n: 1073
min: 1.1102e-16 - max: 2.9061e+153
μ ± 2σ: 1.0371e+149 ± 3.0517e+151
Quartiles - 1st: 1.7462e-01 - 2nd: 3.0158e-01 - 3rd: 5.0127e-01

Welch t-test:
t: -1.3314 - p: .410
############
train_scores
############
Summary for a:
n: 887
min: -1.0000e+00 - max: 1.0000e+00
μ ± 2σ: 5.4695e-01 ± 9.6442e-03
Quartiles - 1st: 4.4577e-02 - 2nd: 6.6760e-01 - 3rd: 9.4845e-01

Summary for b:
n: 791
min: -1.0000e+00 - max: 1.0000e+00
μ ± 2σ: 5.9063e-01 ± 1.6218e-02
Quartiles - 1st: 2.3345e-01 - 2nd: 6.7216e-01 - 3rd: 9.4939e-01

Welch t-test:
t: -14.6843 - p: <.001
########
val_l

In [204]:
using_relu = df['activation_func'] == 'LeakyReLU'

stat_test(masks=(shuffles_batch & using_regression & using_relu, no_shuffling & using_regression & using_relu), 
          test_name='Regression using ReLU Activation - Shuffling Batches vs. Not Shuffling Batches:')

********************************************************************************
Regression using ReLU Activation - Shuffling Batches vs. Not Shuffling Batches:
********************************************************************************
##########
train_loss
##########
Summary for a:
n: 517
min: 2.6007e-14 - max: 2.7440e+152
μ ± 2σ: 1.3287e+148 ± 3.6768e+150
Quartiles - 1st: 2.0113e-01 - 2nd: 3.2423e-01 - 3rd: 5.0204e-01

Summary for b:
n: 519
min: 1.1102e-16 - max: 2.9061e+153
μ ± 2σ: 2.0791e+149 ± 4.3211e+151
Quartiles - 1st: 1.8675e-01 - 2nd: 3.1244e-01 - 3rd: 5.0109e-01

Welch t-test:
t: -1.3311 - p: .410
############
train_scores
############
Summary for a:
n: 427
min: -1.0000e+00 - max: 1.0000e+00
μ ± 2σ: 5.4672e-01 ± 1.3455e-02
Quartiles - 1st: 1.4938e-01 - 2nd: 6.5393e-01 - 3rd: 9.4019e-01

Summary for b:
n: 386
min: -1.0000e+00 - max: 1.0000e+00
μ ± 2σ: 5.7362e-01 ± 2.6014e-02
Quartiles - 1st: 2.4996e-01 - 2nd: 6.5455e-01 - 3rd: 9.3857e-01

Welch t-test:
t: -6.5395 - p: <

In [205]:
using_sigmoid = df['activation_func'] == 'Sigmoid'

stat_test(masks=(shuffles_batch & using_regression & using_sigmoid, 
                 no_shuffling & using_regression & using_sigmoid), 
          test_name='Regression using Sigmoid Activation - Shuffling Batches vs. Not Shuffling Batches:')

********************************************************************************
Regression using Sigmoid Activation - Shuffling Batches vs. Not Shuffling Batches:
********************************************************************************
##########
train_loss
##########
Summary for a:
n: 560
min: 7.2164e-16 - max: 2.1414e+00
μ ± 2σ: 3.1824e-01 ± 8.9017e-03
Quartiles - 1st: 1.4724e-01 - 2nd: 2.9819e-01 - 3rd: 5.0208e-01

Summary for b:
n: 554
min: 4.4409e-16 - max: 1.9929e+00
μ ± 2σ: 3.1560e-01 ± 7.6206e-03
Quartiles - 1st: 1.5110e-01 - 2nd: 2.8748e-01 - 3rd: 5.0145e-01

Welch t-test:
t: 1.5599 - p: .118
############
train_scores
############
Summary for a:
n: 460
min: -1.0000e+00 - max: 1.0000e+00
μ ± 2σ: 5.4718e-01 ± 1.3379e-02
Quartiles - 1st: 3.0323e-03 - 2nd: 6.8760e-01 - 3rd: 9.5232e-01

Summary for b:
n: 405
min: -1.0000e+00 - max: 1.0000e+00
μ ± 2σ: 6.0757e-01 ± 1.2087e-02
Quartiles - 1st: 1.5360e-01 - 2nd: 7.0083e-01 - 3rd: 9.5554e-01

Welch t-test:
t: -14.0702 - p: <.00

# Shuffling Batches in Regression Tasks
It is difficult to comment on whether or not there is any improvement in loss since the results are not statistically significant. However, there is statistically significant evidence that shuffling batches negatively influences training scores, while also improving generalisation ability (validation scores).

In [207]:
using_classification = df['clf_type'] == 'MLPClassifier'

stat_test(masks=(shuffles_batch & using_classification, no_shuffling & using_classification), 
          test_name='Classification - Shuffling Batches vs. Not Shuffling Batches:')

********************************************************************************
Classification - Shuffling Batches vs. Not Shuffling Batches:
********************************************************************************
##########
train_loss
##########
Summary for a:
n: 964
min: 6.6351e-03 - max: 5.1808e+01
μ ± 2σ: 1.4443e+00 ± 6.5767e-01
Quartiles - 1st: 4.6527e-01 - 2nd: 6.9411e-01 - 3rd: 8.5949e-01

Summary for b:
n: 969
min: 8.1027e-03 - max: 4.4671e+01
μ ± 2σ: 1.5245e+00 ± 7.2667e-01
Quartiles - 1st: 4.5403e-01 - 2nd: 6.9411e-01 - 3rd: 9.1865e-01

Welch t-test:
t: -3.7916 - p: <.001
############
train_scores
############
Summary for a:
n: 964
min: 0.0000e+00 - max: 1.0000e+00
μ ± 2σ: 7.2481e-01 ± 6.7188e-03
Quartiles - 1st: 5.0000e-01 - 2nd: 7.5404e-01 - 3rd: 9.7656e-01

Summary for b:
n: 972
min: 0.0000e+00 - max: 1.0000e+00
μ ± 2σ: 7.1432e-01 ± 7.1212e-03
Quartiles - 1st: 5.0000e-01 - 2nd: 7.5000e-01 - 3rd: 9.7539e-01

Welch t-test:
t: 5.7401 - p: <.001
########
val_loss
###

In [208]:
stat_test(masks=(shuffles_batch & using_classification & using_relu, 
                 no_shuffling & using_classification & using_relu), 
          test_name='Classification using ReLU Activation - Shuffling Batches vs. Not Shuffling Batches:')

********************************************************************************
Classification using ReLU Activation - Shuffling Batches vs. Not Shuffling Batches:
********************************************************************************
##########
train_loss
##########
Summary for a:
n: 480
min: 6.6351e-03 - max: 5.1808e+01
μ ± 2σ: 1.7015e+00 ± 9.8321e-01
Quartiles - 1st: 4.3392e-01 - 2nd: 6.9319e-01 - 3rd: 8.1937e-01

Summary for b:
n: 483
min: 8.1027e-03 - max: 4.4671e+01
μ ± 2σ: 1.8547e+00 ± 1.0712e+00
Quartiles - 1st: 4.4635e-01 - 2nd: 6.9346e-01 - 3rd: 9.0703e-01

Welch t-test:
t: -3.8645 - p: <.001
############
train_scores
############
Summary for a:
n: 480
min: 0.0000e+00 - max: 1.0000e+00
μ ± 2σ: 7.5303e-01 ± 1.0009e-02
Quartiles - 1st: 5.0000e-01 - 2nd: 8.7500e-01 - 3rd: 9.8333e-01

Summary for b:
n: 486
min: 0.0000e+00 - max: 1.0000e+00
μ ± 2σ: 7.3373e-01 ± 1.0651e-02
Quartiles - 1st: 5.0000e-01 - 2nd: 8.4531e-01 - 3rd: 9.8346e-01

Welch t-test:
t: 7.7434 - p: <.001

In [209]:
stat_test(masks=(shuffles_batch & using_classification & using_sigmoid, 
                 no_shuffling & using_classification & using_sigmoid), 
          test_name='Classification using Sigmoid Activation - Shuffling Batches vs. Not Shuffling Batches:')

********************************************************************************
Classification using Sigmoid Activation - Shuffling Batches vs. Not Shuffling Batches:
********************************************************************************
##########
train_loss
##########
Summary for a:
n: 484
min: 1.3374e-02 - max: 9.6380e+00
μ ± 2σ: 1.1862e+00 ± 8.1926e-02
Quartiles - 1st: 4.9731e-01 - 2nd: 6.9466e-01 - 3rd: 9.4391e-01

Summary for b:
n: 486
min: 9.9160e-03 - max: 9.0202e+00
μ ± 2σ: 1.1937e+00 ± 1.0534e-01
Quartiles - 1st: 4.5220e-01 - 2nd: 6.9446e-01 - 3rd: 9.6443e-01

Welch t-test:
t: -0.5336 - p: .593
############
train_scores
############
Summary for a:
n: 484
min: 0.0000e+00 - max: 1.0000e+00
μ ± 2σ: 6.9651e-01 ± 1.0900e-02
Quartiles - 1st: 5.0000e-01 - 2nd: 7.4727e-01 - 3rd: 9.7500e-01

Summary for b:
n: 486
min: 0.0000e+00 - max: 1.0000e+00
μ ± 2σ: 6.9483e-01 ± 9.8338e-03
Quartiles - 1st: 5.0000e-01 - 2nd: 7.3359e-01 - 3rd: 9.7500e-01

Welch t-test:
t: 0.6309 - p: .52

# Shuffling Batches in Classification Tasks

With the exception of the training metrics when using the sigmoid activation, we see statistically significant results. There is a much clearer trend when it comes to classification, we see consistent improvement in all metrics, with more marked improvement in validation performance.